# TinyAleph: Semantic Types and Inference

This notebook explores the semantic type system:

- **Prime-indexed terms**: Nouns, Adjectives, Chains, Fusions
- **Well-formedness**: Type checking rules
- **Reduction semantics**: Normalizing terms
- **Semantic inference**: Reasoning with primes

In [ ]:
import sys
sys.path.insert(0, '..')

from tinyaleph.semantic.types import (
    NounTerm, AdjTerm, ChainTerm, FusionTerm, SentenceTerm, TypeChecker
)
from tinyaleph.semantic.reduction import (
    NextPrimeOperator, ModularPrimeOperator, ReductionSystem
)
from tinyaleph.semantic.lambda_calc import (
    LambdaTranslator, ConceptInterpreter, PRQS_LEXICON
)
from tinyaleph.semantic.inference import (
    SemanticInference, CompoundBuilder, SEMANTIC_PRIMES
)
from tinyaleph.core.primes import is_prime, nth_prime, first_n_primes

print("TinyAleph Semantic Type System")

## 1. Prime-Indexed Terms

The type system uses primes to index semantic entities:

- **N(p)**: Noun term indexed by prime p
- **A(p)**: Adjective term indexed by prime p
- **ChainTerm**: A(p₁)...A(pₖ)N(q) composition
- **FusionTerm**: FUSE(p, q, r) triadic fusion

In [ ]:
# Noun terms - entities indexed by primes
entity_7 = NounTerm(7)
entity_11 = NounTerm(11)
entity_13 = NounTerm(13)

print("Noun terms (entities):")
print(f"  N(7):  prime={entity_7.p}, well-formed={entity_7.is_well_formed()}")
print(f"  N(11): prime={entity_11.p}, well-formed={entity_11.is_well_formed()}")
print(f"  N(13): prime={entity_13.p}, well-formed={entity_13.is_well_formed()}")

In [ ]:
# Adjective terms - properties/operators indexed by primes
adj_2 = AdjTerm(2)
adj_3 = AdjTerm(3)
adj_5 = AdjTerm(5)

print("Adjective terms (properties):")
print(f"  A(2): prime={adj_2.p}")
print(f"  A(3): prime={adj_3.p}")
print(f"  A(5): prime={adj_5.p}")

## 2. Chain Terms and Well-Formedness

A chain term A(p₁)...A(pₖ)N(q) is **well-formed** if and only if p_i < q for all i.

This ensures adjectives "fit" the noun they modify.

In [ ]:
# Well-formed chain: all adjective primes < noun prime
chain_good = ChainTerm(
    adjectives=[AdjTerm(2), AdjTerm(3), AdjTerm(5)],
    noun=NounTerm(11)  # 2, 3, 5 < 11 ✓
)

# Ill-formed chain: adjective prime >= noun prime
chain_bad = ChainTerm(
    adjectives=[AdjTerm(2), AdjTerm(7), AdjTerm(13)],
    noun=NounTerm(11)  # 13 > 11 ✗
)

print("Chain terms:")
print(f"  A(2)A(3)A(5)N(11): well-formed = {chain_good.is_well_formed()}")
print(f"  A(2)A(7)A(13)N(11): well-formed = {chain_bad.is_well_formed()}")

In [ ]:
# Type checker
checker = TypeChecker()

terms = [
    ("N(7)", NounTerm(7)),
    ("A(3)N(7)", ChainTerm([AdjTerm(3)], NounTerm(7))),
    ("A(11)N(7)", ChainTerm([AdjTerm(11)], NounTerm(7))),  # 11 > 7, ill-formed
]

print("Type checking:")
for name, term in terms:
    valid = checker.check(term)
    status = "✓" if valid else "✗"
    print(f"  {name}: {status}")

## 3. Fusion Terms

A fusion term FUSE(p, q, r) is **well-formed** if p + q + r is prime.

In [ ]:
# Find valid fusions
print("Fusion terms FUSE(p, q, r) where p+q+r is prime:\n")

valid_fusions = []
primes = first_n_primes(6)

for p in primes:
    for q in primes:
        for r in primes:
            fusion = FusionTerm(p, q, r)
            if fusion.is_well_formed():
                valid_fusions.append((p, q, r, fusion.fused_value()))

# Show first 10
for p, q, r, s in valid_fusions[:10]:
    print(f"  FUSE({p}, {q}, {r}) = {s} (prime)")

## 4. Prime-Preserving Operators

Operators that transform primes into primes.

In [ ]:
# NextPrime operator: p → next_prime(p)
next_op = NextPrimeOperator()

print("NextPrime operator ⊕:")
for p in [2, 3, 5, 7, 11, 13]:
    result = next_op.apply(p)
    print(f"  ⊕({p}) = {result}")

In [ ]:
# Modular operator: cycles through primes
mod_op = ModularPrimeOperator(k=3, n=10)

print("Modular operator (offset=3, mod 10):")
for p in [2, 3, 5, 7, 11]:
    result = mod_op.apply(p)
    print(f"  op({p}) = {result}")

## 5. PRQS Lexicon - Semantic Primes

The library maps primes to semantic concepts.

In [ ]:
# Display PRQS lexicon
print("PRQS Lexicon - Core Semantic Primes:\n")
print(f"{'Prime':>6} {'Concept':<15} {'Category':<12}")
print("-" * 35)

for prime, info in sorted(PRQS_LEXICON.items())[:15]:
    print(f"{prime:>6} {info['name']:<15} {info['category']:<12}")

In [ ]:
# Interpret primes as concepts
interpreter = ConceptInterpreter()

print("Concept interpretation:")
for p in [2, 3, 5, 7, 11, 13]:
    concept = interpreter.interpret(p)
    print(f"  Prime {p}: {concept}")

## 6. Compound Building

Build compound concepts from primes using logical operations.

In [ ]:
builder = CompoundBuilder()

# Conjunction: A ∧ B = a × b
print("Conjunction (A ∧ B = a × b):")
for a, b in [(2, 3), (3, 5), (5, 7)]:
    compound = builder.build_and(a, b)
    print(f"  {a} ∧ {b} = {compound}")

# Negation: ¬A = next_prime(a)
print("\nNegation (¬A = next_prime(a)):")
for a in [2, 3, 5, 7]:
    neg = builder.build_not(a)
    print(f"  ¬{a} = {neg}")

## 7. NSM Semantic Primes

Based on Wierzbicka's Natural Semantic Metalanguage.

In [ ]:
# Display NSM semantic primes
print("Natural Semantic Metalanguage Primes:\n")

categories = {}
for word, prime in SEMANTIC_PRIMES.items():
    # Group by first letter as rough category
    cat = word[0].upper()
    if cat not in categories:
        categories[cat] = []
    categories[cat].append((word, prime))

# Show first 20
items = list(SEMANTIC_PRIMES.items())[:20]
for word, prime in items:
    print(f"  '{word}' → {prime}")

## Summary

- **NounTerm N(p)**: Entities indexed by primes
- **AdjTerm A(p)**: Properties indexed by primes  
- **ChainTerm**: Well-formed when adjective primes < noun prime
- **FusionTerm**: Well-formed when sum is prime
- **Operators**: Transform primes to primes
- **PRQS Lexicon**: Maps primes to semantic concepts
- **NSM Primes**: Universal semantic primitives